In [2]:
import pandas as pd
from datetime import datetime
import re

FILE = "D:/AWIS/Semester_03/SFP/Webscraper_Autoscout/SFP_Workspace/car_data_csv.csv"

In [ ]:
# Bereinigung der Daten in der Spalte column_name um value
# isRegex gibt an, ob value ein regulärer Ausdruck ist
def replace_by_header(column_name, filename, value, isRegex):      
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].replace(value, '', regex=isRegex)
    df.to_csv(filename, index = False, sep=';')

# Berenigung der Spalten Schadstoffklasse, CO₂-Emissionen2 und Leistung zu numerischen Werten
replace_by_header("Schadstoffklasse", FILE, "Euro", False)
replace_by_header("CO₂-Emissionen2",FILE, "g/(komb.)", False)
replace_by_header("Leistung",FILE, r"\(\d+PS\)", True)

In [ ]:
# Berenigung kann nur gestartet werden wenn die Spalten im Fromat 'MM.YYYY' oder 'DD.MM.YYYY' vorhanden sind
# vereinheitlicht die Spalten Erstzulassung und HU zur Anzahl der Monate oder Jahre
def clean_column(column_name, filename):
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].apply(lambda x: 24 if x == 'Neu' else (berechne_differenz(x, True) if re.match(r"\d{2}/\d{4}", x) else berechne_differenz(x, False)))
    df.to_csv(filename, index = False, sep=';')

# berechnet die Differenz zwischen dem aktuellen Datum und dem übergebenen Datum
def berechne_differenz(datum, monate):
    datum = pd.to_datetime(datum)
    aktuelles_datum = datetime.now()  
    if monate:
        differenz = (datum.year - aktuelles_datum.year) * 12 + datum.month - aktuelles_datum.month
    else:
        differenz = aktuelles_datum.year - datum.year
    return differenz

# Vereinheitlichen der Spalten Erstzulassung 
clean_column("Erstzulassung", FILE)
clean_column("HU", FILE)

In [3]:
# ersetzt alle Werte, die in der Spalte column_name enthalten sind, durch new_value, wenn substring in dem Wert enthalten ist
def replace_by_header_contains(column_name, filename, substring, new_value):
    df = pd.read_csv(filename, sep=';')
    if column_name in df.columns:
        df[column_name] = df[column_name].apply(lambda x: new_value if substring in str(x) else x)
    df.to_csv(filename, index=False, sep=';')

# Vereinheitlichen der Kraftstoffe in die Klassen Benzin, Diesel und Gas
replace_by_header_contains("Kraftstoff", FILE, "Benzin", "Benzin")
replace_by_header_contains("Kraftstoff", FILE, "Super", "Benzin")
replace_by_header_contains("Kraftstoff", FILE, "Diesel", "Diesel")
replace_by_header_contains("Kraftstoff", FILE, "gas", "Gas")

# Vereinheitlichen der Schadstoffklassen 6, 6c, 6d und 6-d-TEMP zur Klasse 6
replace_by_header_contains("Schadstoffklasse", FILE, "d", "6")
replace_by_header_contains("Schadstoffklasse", FILE, "c", "6")

# Vereinheitlichen der Variable Anzahl Türen -> 3/5 Türer
replace_by_header_contains("Türen", FILE, "2", "3")
replace_by_header_contains("Türen", FILE, "4", "5")

#Alle Fahrzeuge ohne einer aktuellen Hauptuntersuchung werden auf 0 gesetzt -> 0 representiert abgelaufene HU
replace_by_header_contains("HU", FILE, "-", "0")